In [ ]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=200,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
    tf.app.run()

/home/ayrus/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4bb594aef0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorf

INFO:tensorflow:loss = 2.29836, step = 2
INFO:tensorflow:probabilities = [[ 0.09493149  0.09159368  0.10541724  0.10726168  0.09000868  0.10365473
   0.08901907  0.11697116  0.0999189   0.10122327]
 [ 0.09475148  0.08476356  0.10473465  0.10269079  0.10820507  0.10205277
   0.1023953   0.10335118  0.09932145  0.09773378]
 [ 0.09666377  0.09189386  0.11184832  0.12525629  0.08803128  0.10613602
   0.08770281  0.09919422  0.09846776  0.09480566]
 [ 0.09511704  0.08323969  0.10492295  0.12046062  0.0899155   0.10943109
   0.10245151  0.10481887  0.0951535   0.09448923]
 [ 0.090534    0.08438878  0.10305683  0.10564437  0.11381219  0.09882434
   0.09418075  0.10654178  0.10788479  0.09513218]
 [ 0.10055796  0.09275862  0.11174862  0.10607776  0.10045106  0.0994854
   0.09604444  0.10546669  0.09576417  0.09164521]
 [ 0.0865353   0.08987972  0.10055377  0.11086425  0.09012554  0.10609794
   0.10159233  0.11427844  0.09191825  0.10815443]
 [ 0.10479596  0.07166369  0.10551837  0.09717213  0.

INFO:tensorflow:global_step/sec: 1.23364
INFO:tensorflow:probabilities = [[ 0.10466193  0.08948147  0.10595531  0.10246599  0.09725314  0.1016527
   0.09189203  0.10315661  0.11191983  0.091561  ]
 [ 0.1009638   0.10260421  0.09336922  0.10900597  0.1032108   0.10099617
   0.10422006  0.10321277  0.0968876   0.08552934]
 [ 0.1046268   0.08436612  0.11866575  0.10444701  0.09607717  0.09779739
   0.09349915  0.11022119  0.09825619  0.09204324]
 [ 0.10473091  0.09116936  0.10842418  0.10454322  0.09804547  0.10243167
   0.09247138  0.10759012  0.09781985  0.09277388]
 [ 0.10181768  0.08787084  0.11148959  0.09789453  0.09517673  0.11432619
   0.09939503  0.10501594  0.10388336  0.08313014]
 [ 0.09520501  0.08530067  0.1088793   0.10702955  0.08281336  0.10630873
   0.08938256  0.12094287  0.10082553  0.1033124 ]
 [ 0.09811918  0.09300403  0.09731568  0.10131812  0.10179431  0.10537334
   0.1001361   0.0988127   0.11176031  0.09236629]
 [ 0.10213183  0.08482904  0.12019859  0.09821025  0.

INFO:tensorflow:loss = 2.29415, step = 102 (81.140 sec)
INFO:tensorflow:probabilities = [[ 0.09653919  0.0793862   0.10657249  0.10961174  0.10959847  0.09120948
   0.09721421  0.10983822  0.10458799  0.09544201]
 [ 0.08132895  0.08615614  0.10169288  0.09711879  0.10594404  0.1090758
   0.10007009  0.11319386  0.1049218   0.10049765]
 [ 0.09410964  0.07472464  0.09850023  0.104459    0.09613773  0.11191215
   0.09014159  0.1228242   0.10404789  0.10314292]
 [ 0.10901465  0.09175946  0.104268    0.10486243  0.09419911  0.09266914
   0.09801096  0.11154687  0.09693502  0.09673435]
 [ 0.09295953  0.08864411  0.10571045  0.10989548  0.09583641  0.10123802
   0.09068472  0.11206837  0.10936831  0.09359458]
 [ 0.10060109  0.0928512   0.10327027  0.10132476  0.09565412  0.09410724
   0.09841818  0.10961404  0.10977142  0.09438765]
 [ 0.10817721  0.09072025  0.10909729  0.12016357  0.09637434  0.09753712
   0.10342015  0.08899417  0.10001022  0.08550569]
 [ 0.09623374  0.0846618   0.09743068 